In [10]:
import torch

In [11]:
from torch import nn

In [12]:
from torchinfo import summary

In [23]:
class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
                                  ,nn.BatchNorm2d(out_channels)
                                  ,nn.ReLU(inplace=True))
        

    def forward(self, x):
        x = self.conv(x)
        return x

In [25]:
BasicConv2d(3,5,kernel_size=3)# kernel_size是卷积层内核数 填入了一次kernel_size会代入**Kwargs，后面调用就不用了必填了

BasicConv2d(
  (conv): Sequential(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
)

In [15]:
# 所有的bias都是false，所有的层都没有偏差。可能有一些可选的超参数，用**kwargs代替，比如

In [258]:
class Inception(nn.Module):

    def __init__(self,in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj,
                ):
        super().__init__()
        self.branch1 = BasicConv2d(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            BasicConv2d(in_channels, ch3x3red, kernel_size=1),
            BasicConv2d(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            BasicConv2d(in_channels, ch5x5red, kernel_size=1),
            # Here, kernel_size=3 instead of kernel_size=5 is a known bug.
            # Please see https://github.com/pytorch/vision/issues/906 for details.
            BasicConv2d(ch5x5red, ch5x5, kernel_size=5, padding=2)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv2d(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)


In [260]:
inception3a = Inception(192,64,96,128,16,32,32)

In [262]:
data = torch.ones(30,192,28,28)

In [264]:
inception3a(data).shape

torch.Size([30, 256, 28, 28])

In [270]:
class AuxClf(nn.Module):
    def __init__(self
                 ,in_channels
                 ,num_classes
                 ,**Kwargs):
        super().__init__()
        self.feature_=nn.Sequential(nn.AvgPool2d(kernel_size=5,stride=3)
                                    ,BasicConv2d(in_channels,128,kernel_size=1))
        self.clf_=nn.Sequential(nn.Linear(4*4*128,1024)
                                ,nn.ReLU(inplace=True)
                                ,nn.Dropout(0.7)
                                ,nn.Linear(1024,num_classes))
    def forward(self,x):
            x=self.feature_(x)
            x=x.view(-1,4*4*128)
            x=self.clf_(x)
            return x

In [272]:
# 4a

In [274]:
AuxClf(512,1000)

AuxClf(
  (feature_): Sequential(
    (0): AvgPool2d(kernel_size=5, stride=3, padding=0)
    (1): BasicConv2d(
      (conv): Sequential(
        (0): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
  )
  (clf_): Sequential(
    (0): Linear(in_features=2048, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.7, inplace=False)
    (3): Linear(in_features=1024, out_features=1000, bias=True)
  )
)

In [276]:
class GoogLeNet(nn.Module):
     def __init__(self,num_classes=1000):
        super().__init__()
        self.conv1 = BasicConv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)
        self.conv2 = BasicConv2d(64, 64, kernel_size=1)
        self.conv3 = BasicConv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)
       

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)

        self.aux1 = AuxClf(512, num_classes)
        self.aux2 = AuxClf(528, num_classes)
         
     def forward(self,x):
        # type: (Tensor) -> Tuple[Tensor, Optional[Tensor], Optional[Tensor]]
        # N x 3 x 224 x 224
        # x = self.conv1(x)
        # N x 64 x 112 x 112
        x = self.maxpool1(self.conv1(x))
        
        # N x 64 x 56 x 56
        # x = self.conv2(x)
        # N x 64 x 56 x 56
        x = self.maxpool2(self.conv3(self.conv2(x)))
        # N x 192 x 56 x 56
        # x = self.maxpool2(x)

        # N x 192 x 28 x 28
        x = self.inception3a(x)
        
        # N x 256 x 28 x 28
        # x = self.inception3b(x)
        # N x 480 x 28 x 28
        x = self.maxpool3(self.inception3b(x))

        
        # N x 480 x 14 x 14
        x = self.inception4a(x)
        # N x 512 x 14 x 14
        
        aux_1 = self.aux1(x)

        x = self.inception4b(x)
        # N x 512 x 14 x 14
        x = self.inception4c(x)
        # N x 512 x 14 x 14
        x = self.inception4d(x)
        # N x 528 x 14 x 14

        
        aux_2 = self.aux2(x)

       
        # N x 832 x 14 x 14
        x = self.maxpool4(self.inception4e(x))
        
        # N x 832 x 7 x 7
        x = self.inception5a(x)
        # N x 832 x 7 x 7
        x = self.inception5b(x)
        # N x 1024 x 7 x 7

        x = self.avgpool(x)
        # N x 1024 x 1 x 1
        x = torch.flatten(x, 1)
        # N x 1024
        x = self.dropout(x)
        x = self.fc(x)
        # N x 1000 (num_classes)
        
        return x, aux_1, aux_2


In [278]:
data = torch.ones(20,3,224,224)

In [280]:
net = GoogLeNet()

In [282]:
x,aux1,aux2 = net(data)

In [283]:
for i in [x,aux1,aux2]:
    print(i.shape)

torch.Size([20, 1000])
torch.Size([20, 1000])
torch.Size([20, 1000])


In [287]:
summary(net,(20,3,224,224),device="cpu")

Layer (type:depth-idx)                        Output Shape              Param #
GoogLeNet                                     [20, 1000]                --
├─BasicConv2d: 1-1                            [20, 64, 112, 112]        --
│    └─Sequential: 2-1                        [20, 64, 112, 112]        --
│    │    └─Conv2d: 3-1                       [20, 64, 112, 112]        9,408
│    │    └─BatchNorm2d: 3-2                  [20, 64, 112, 112]        128
│    │    └─ReLU: 3-3                         [20, 64, 112, 112]        --
├─MaxPool2d: 1-2                              [20, 64, 56, 56]          --
├─BasicConv2d: 1-3                            [20, 64, 56, 56]          --
│    └─Sequential: 2-2                        [20, 64, 56, 56]          --
│    │    └─Conv2d: 3-4                       [20, 64, 56, 56]          4,096
│    │    └─BatchNorm2d: 3-5                  [20, 64, 56, 56]          128
│    │    └─ReLU: 3-6                         [20, 64, 56, 56]          --
├─BasicConv2